In [129]:
import osmnx as ox
import networkx as nx
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

#Graph Algorithms

def get_odd_nodes(graph):
    odd_nodes = []
    for i,j in nx.degree(graph):
        if j%2==1:
            odd_nodes.append(i)
    return odd_nodes

def node_combo(odd_list):
    return list(itertools.combinations(odd_list, 2))

def get_shortest_paths(graph, pairs):
    short_paths = {}
    for pair in pairs:
        short_paths[pair] = nx.dijkstra_path_length(graph, pair[0], pair[1], weight='distance')
    return short_paths

def make_complete(pair_weights):
    graph = nx.Graph()
    for k, v in pair_weights.items():
        wt_i = - v
        graph.add_edge(k[0], k[1], attr_dict={'distance': v, 'weight': wt_i})
    return graph

def min_weight_matching(complete_odd_graph):
    odd_max_match = nx.algorithms.max_weight_matching(complete_odd_graph, True)
    odd_min_match = list(pd.unique([tuple(sorted([k, v])) for k, v in odd_max_match]))
    return odd_min_match

def join_aug_edges(graph,weight_pairs):
    graph_aug = nx.MultiGraph(graph.copy())
    for pair in weight_pairs:
        graph_aug.add_edge(pair[0],
                           pair[1],
                           attr_dict={'distance': nx.dijkstra_path_length(graph, pair[0], pair[1]),
                                      'trail': 'augmented'}
                          )
    return graph_aug

def create_eulerian_circuit(graph_augmented, graph_original, starting_node=None):
    euler_circuit = []
    naive_circuit = list(nx.eulerian_circuit(graph_augmented, source=starting_node))

    for edge in naive_circuit:
        edge_data = graph_augmented.get_edge_data(edge[0], edge[1])
        if "attr_dict" in edge_data[0] and edge_data[0]['attr_dict']['trail'] != 'augmented':
            # If `edge` exists in original graph, grab the edge attributes and add to eulerian circuit.
            edge_att = graph_original[edge[0]][edge[1]]
            euler_circuit.append((edge[0], edge[1], edge_att))
        else:
            aug_path = nx.shortest_path(graph_original, edge[0], edge[1], weight='distance')
            aug_path_pairs = list(zip(aug_path[:-1], aug_path[1:]))

            # print('Filling in edges for augmented edge: {}'.format(edge))
            # print('Augmenting path: {}'.format(' => '.join(str(aug_path))))
            # print('Augmenting path pairs: {}\n'.format(aug_path_pairs))

            # If `edge` does not exist in original graph, find the shortest path between its nodes and
            #  add the edge attributes for each link in the shortest path.
            for edge_aug in aug_path_pairs:
                edge_aug_att = graph_original[edge_aug[0]][edge_aug[1]]
                euler_circuit.append((edge_aug[0], edge_aug[1], edge_aug_att))

    return euler_circuit

In [130]:
#Map Utilities

def get_node_position(G):
    return {node[0]: (node[1]['x'], -node[1]['y']) for node in G.nodes(data=True)}
            
def euler_circuit_to_route(euler_circuit):
    route = []
    for edge in euler_circuit:
        route.append(edge[0])
    return route


def route_to_long_lat(G, route):
    long = []
    lat = []
    for i in route:
        point = G.nodes[i]
        long.append(point['x'])
        lat.append(point['y'])
    return long, lat


def long_lat_to_points(long, lat):
    origin_point = long[0], lat[0]
    dest_point = long[-1], lat[-1]
    return origin_point, dest_point

def get_first_element_from_multi_edge_graph(multi_edge_graph):
  
    for e in multi_edge_graph:
        return e[0]


In [131]:
#Plot functions


def plot_path(lat, long, origin_point, destination_point):
    fig = go.Figure(go.Scattermapbox(
        name="Path",
        mode="lines",
        lon=long,
        lat=lat,
        marker={'size': 10},
        line=dict(width=4.5, color='blue')))

    lat_center = np.mean(lat)
    long_center = np.mean(long)

    fig.update_layout(mapbox_style="stamen-terrain",
                      mapbox_center_lat=30, mapbox_center_lon=-80)
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0},
                      mapbox={
                          'center': {'lat': lat_center,
                                     'lon': long_center},
                          'zoom': 13})
    fig.show()
    

def create_cpp_edgelist(euler_circuit):
    """
    Create the edgelist without parallel edge for the visualization
    Combine duplicate edges and keep track of their sequence and # of walks
    Parameters:
        euler_circuit: list[tuple] from create_eulerian_circuit
    """
    cpp_edgelist = {}

    for i, e in enumerate(euler_circuit):
        edge = frozenset([e[0], e[1]])

        if edge in cpp_edgelist:
            cpp_edgelist
    return list(cpp_edgelist.values())
def make_circuit_video(image_path, movie_filename, fps=5):
   
    filenames = glob.glob(image_path + 'img*.png')
    filenames_sort_indices = np.argsort([int(os.path.basename(filename).split('.')[0][3:]) for filename in filenames])
    filenames = [filenames[i] for i in filenames_sort_indices]


    with imageio.get_writer(movie_filename, mode='I', fps=fps) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

In [132]:
def main():
    G = ox.graph_from_point((47.5837984, -52.7126917), dist=1000, network_type='drive')
    G = ox.utils_graph.get_undirected(G)
    
  
    #find odd degree Nodes: odd_nodes() returns list
    #computes all nodes with odd degree in the graph object
    odd_nodes = get_odd_nodes(G)
    
    #computing odd combos: node_combo() returns list
    #computes and return a list containing 2-pair combinations of odd nodes
    odd_combo = node_combo(odd_nodes)
    
    #Computing shortest paths between all odd degree vertices
    # : get_shortest_paths() returns dict
    odd_pair_shortest_paths = get_shortest_paths(G,odd_combo)
    
    #joining odd vertex to every other vertex to make a complete graph
    # :make_complete() returns graph
    odd_complete_graph = make_complete(odd_pair_shortest_paths)
    
    #Matching odd vertices to their closest odd neighbour
    # min_weight_match(): returns list
    odd_min_match = min_weight_matching(odd_complete_graph)
    
    #adding odd_min_matches edges to graph G
    g_aug = join_aug_edges(G,odd_min_match)
    
    s = g_aug.edges()
    source_s = get_first_element_from_multi_edge_graph(s)
    euler_circuit = create_eulerian_circuit(g_aug, G, source_s)
    
    route = euler_circuit_to_route(euler_circuit)
    
    long, lat = route_to_long_lat(G, route)
    origin_point, dest_point = long_lat_to_points(long, lat)
    print("Plotting the route")
    # Plot the route
    plot_path(lat, long, origin_point, dest_point)
    
    
    


if __name__ == '__main__':
    main()

Plotting the route
